<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Feature Engineering</a></span></li><li><span><a href="#Imbalanced-data" data-toc-modified-id="Imbalanced-data-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Imbalanced data</a></span></li><li><span><a href="#Import" data-toc-modified-id="Import-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-0.5"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Load data</a></span></li></ul></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#Create-categorical-variables" data-toc-modified-id="Create-categorical-variables-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Create categorical variables</a></span></li><li><span><a href="#Split-train-and-test-set" data-toc-modified-id="Split-train-and-test-set-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Split train and test set</a></span></li><li><span><a href="#Encoding-Categorical-Variables" data-toc-modified-id="Encoding-Categorical-Variables-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Encoding Categorical Variables</a></span></li><li><span><a href="#Feature-Scaling---StandardScaler" data-toc-modified-id="Feature-Scaling---StandardScaler-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Feature Scaling - StandardScaler</a></span></li></ul></li><li><span><a href="#Building-a-model" data-toc-modified-id="Building-a-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Building a model</a></span></li></ul></div>

# Bank Marketing Builing Models

### Introduction

__The main goal of this project is to predict if the client will subscribe a term deposit (variable y).__ I will show how to approach developing a model to predict client's subscriptions using an imbalanced dataset. 

The dataset is provided by [UCI-Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/bank+marketing). The data contains direct marketing campaigns (phone calls) of a Portuguese banking institution. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be 'yes' or 'no' subscribed. 


### Feature Engineering
The original dataset has 10 numerical variables (9 continuous and 1 discrete), 10 categorical variables. I created 2 categorical variables `age_group` and `pdays_group` by using `age` and `pdays`, and removed the original variables. 
Also, I removed `duration` variable due to the given information: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

So, the final dataset has 11 numerical variables and 8 categorical variables. 

Before building a model, we need to encode categorical variables as numerical values. So, I will use integer encoding by using feature engine package. Also, I will use StandardScaler from sklearn to scale all numerical variables. 

I'll show the feature engineering process in this notebook with less detail, so if you want to see more, please take a look at [Bank_Marketing_EDA.ipynb](https://github.com/yejiseoung/BankMarketing/blob/develop/Bank_Marketing_EDA.ipynb) and [Bank_Marketing_FeatureEngineering.ipynb](https://github.com/yejiseoung/BankMarketing/blob/develop/Bank_Marketing_FeatureEngineering.ipynb). You can find the visualizations and more detail in those notebooks. 




### Imbalanced data
We have a classification problem. The target is a binary variable (no-yes), and it shows 89% of no and 11% of yes responses. This dataset is a typical imbalanced data which means that datasets have many more instances of certain classes (no - 89%) than of others (yes - 11%). Since most machine learning algorithms assume balanced distributions, samples from the minority class (in this case, yes label) are likely misclassified. So, we need to deal with this very carefully. Through the notebook, I will implement EasyEnsemble technique which is an ensemble algorithms that were designed to work with imbalanced datasets. 

You might wonder why I won't use under- or over-sampling techniques. I tried all possible techinques to balance imbalanced dataset and none of under- or over-sampling techinques did not improve the model performance. You can find the whole process in [Bank_Marketing_BalancingSampling.ipynb](https://github.com/yejiseoung/BankMarketing/blob/develop/Bank_Marketing_BalancingSampling.ipynb)


- __Reference__: [Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014


I tried to figure out if we need to perform feature selection, but the models did not improve after selecting features, so I will use all variables here. If you want to see the feature selection process, please see [Bank_Marketing_FeatureSelection.ipynb](https://github.com/yejiseoung/BankMarketing/blob/develop/Bank_Marketing_FeatureSelection.ipynb). 

### Import

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

from collections import Counter

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pathlib import Path
import os
os.getcwd()

'/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/BankMarketing'

In [2]:
path = Path('/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/BankMarketing')

In [3]:
# for the model 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    GradientBoostingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier)

from catboost import CatBoostClassifier

from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    recall_score,
    precision_score
)

from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline

# for feature engineering
from feature_engine import encoding as ce


# for cross-validation
from imblearn.pipeline import make_pipeline

# for ensemble sampling
from imblearn.ensemble import EasyEnsembleClassifier

# for feature selection
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# for feature selection
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE

### Load data

In [4]:
data = pd.read_csv(path/'bank.csv', delimiter=';')
print(data.shape)
data.head()

(41188, 21)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
# we have imbalanced data
data['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [6]:
# no missing values
[var for var in data.columns if data[var].isnull().mean() > 0]

[]

## Feature Engineering

### Create categorical variables

Here, I will drop `duration` and create `age_group` and `pdays_group` by using `age` and `pdays` variables, and drop the original variables. 

In [7]:
data.drop('duration', axis=1, inplace=True)
data.shape

(41188, 20)

`age`: 17-30 for young adult, 31-40, 41-50, 51-60, and more than 61

In [8]:
# let's divide age into the buckets 
buckets = [16, 30, 40, 50, 60, 100]

# bucket labels
labels = ['17-30', '31-40', '41-50', '51-60', '>60']

# discretization
data['age_group'] = pd.cut(data['age'], bins=buckets, labels=labels, include_lowest=True)
data['age_group'] = data['age_group'].astype('O') # change dtype

# drop the original age column
data.drop('age', axis=1, inplace=True)

In [9]:
data['age_group'].head()

0    51-60
1    51-60
2    31-40
3    31-40
4    51-60
Name: age_group, dtype: object

`pdays`: 1w (less than 7days), 2w (less than 14 days), and >2w (more than 14days)

In [10]:
# pdays has 999 values which mean that 
# client was not previous contacted. 
# I will change 999 to -1 
data['pdays'] = data['pdays'].replace(999, -1)

# let's divide pdays into 3 groups
bins = [0, 7, 14, 30]

# labels
labels = ['1w', '2w', '>2w']

data['pdays_group'] = pd.cut(data['pdays'], bins=bins, labels=labels, 
                             include_lowest=False) # False makes -1 value to missing values

# fill missing values to Not contacted
data['pdays_group'] = data['pdays_group'].astype('O')
data['pdays_group'].fillna('Not contacted', inplace=True)

# drop the original column
data.drop('pdays', axis=1, inplace=True)

In [11]:
data['pdays_group'].head()

0    Not contacted
1    Not contacted
2    Not contacted
3    Not contacted
4    Not contacted
Name: pdays_group, dtype: object

In [12]:
data.head()

,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,age_group,pdays_group
0,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,51-60,Not contacted
1,services,married,high.school,unknown,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,51-60,Not contacted
2,services,married,high.school,no,yes,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,31-40,Not contacted
3,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,31-40,Not contacted
4,services,married,high.school,no,no,yes,telephone,may,mon,1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,51-60,Not contacted


Change no, yes values to 0, 1 

In [13]:
data['y'] = data['y'].map({'no':0, 'yes': 1})

### Split train and test set

In [14]:
# Create lists for categorical and numerical variables

cat_vars = [var for var in data.columns if var != 'y' and data[var].dtype=='O']
num_vars = [var for var in data.columns if var != 'y' and data[var].dtype!='O']

print('Number of Categorical variables: {}'.format(len(cat_vars)))
print('Number of Numerical variables: {}'.format(len(num_vars)))

Number of Categorical variables: 12
Number of Numerical variables: 7


In [15]:
# separate train and test set
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('y', axis=1),
    data['y'],
    test_size=0.3,
    random_state=0
)

X_train.shape, X_test.shape

((28831, 19), (12357, 19))

### Encoding Categorical Variables

In [16]:
# using OrdinalEncoder from feature-engine

ordinal_enc = ce.OrdinalEncoder(
    encoding_method='arbitrary',
    variables=cat_vars)

ordinal_enc.fit(X_train) 

OrdinalEncoder(encoding_method='arbitrary',
               variables=['job', 'marital', 'education', 'default', 'housing',
                          'loan', 'contact', 'month', 'day_of_week', 'poutcome',
                          'age_group', 'pdays_group'])

In [17]:
X_train = ordinal_enc.transform(X_train)
X_test = ordinal_enc.transform(X_test)

### Feature Scaling - StandardScaler

In [18]:
# set up scaler
scaler = StandardScaler()

# fit the scaler to the train set, it will learn the parameters
scaler.fit(X_train)

# transform train and test set
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Building a model

EasyEnsemble involves creating balanced samples of the training datset by selecting all examples from the minority class and a subset from the majority class. Rather than using pruned decision trees, boosted decision trees are used on each subset, specifically the AdaBoost algorithm. 

The EasyEnsembleClassifier class from the imbalanced-learn library provides an implmentation of the easy ensemble technique. I will use EasyEnsembleClassifier in this notebook. 

We can choose the classifiers to build the best model, so I will use RandomSearchCV to find the best hyperparameters and estimators for this problem. 

In [47]:
def run_RFs(X_train, X_test, y_train, y_test):
    """ Function to evaluate random forests performance"""
    rf = RandomForestClassifier(n_estimators=20)
    rf.fit(X_train, y_train)
    
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('RF roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('RF roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [45]:
def run_easy(X_train, X_test, y_train, y_test):
    """Function to evaluate EasyEnsemble performance"""
    easy = EasyEnsembleClassifier(
        n_estimators=20,
        sampling_strategy='auto', 
        n_jobs=1,
        random_state=42)
    
    easy.fit(X_train, y_train)
    
    print('Train set')
    pred = easy.predict_proba(X_train)
    print('EasyEnsemble roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    
    print('Test set')
    pred = easy.predict_proba(X_test)
    print('EasyEnsemble roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [48]:
# base model for RF
run_RFs(X_train, X_test, y_train, y_test)

Train set
RF roc-auc: 0.9985199423420013
Test set
RF roc-auc: 0.7568164657379993


Random forest shows over-fitting, and we can solve this problem by tuning the hyperparamters, but I won't do it here because I wanted to create base model. 

In [46]:
# base model
run_easy(X_train, X_test, y_train, y_test)

Train set
EasyEnsemble roc-auc: 0.7978208876756269
Test set
EasyEnsemble roc-auc: 0.7941023799583999


In [31]:
# set up classifiers
easy = EasyEnsembleClassifier(random_state=42)
ada = AdaBoostClassifier()
gbm = GradientBoostingClassifier()

# determine the hyperparamter space
param_grid = dict(
    n_estimators=stats.randint(10, 200),
    sampling_strategy = ['float', 'auto'],
    base_estimator=[ada, gbm]
)

In [33]:
search = RandomizedSearchCV(easy,
                           param_grid,
                           scoring='roc_auc',
                           cv=5,
                           n_iter=60,
                           random_state=42,
                           n_jobs=1,
                           refit=True)

search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=EasyEnsembleClassifier(random_state=42),
                   n_iter=60, n_jobs=1,
                   param_distributions={'base_estimator': [AdaBoostClassifier(),
                                                           GradientBoostingClassifier()],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8a596bf850>,
                                        'sampling_strategy': ['float', 'auto']},
                   random_state=42, scoring='roc_auc')

In [34]:
search.best_params_

{'base_estimator': GradientBoostingClassifier(),
 'n_estimators': 98,
 'sampling_strategy': 'auto'}

In [35]:
results = pd.DataFrame(search.cv_results_)
print(results.shape)
results.head()

(60, 16)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_base_estimator,param_n_estimators,param_sampling_strategy,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006823,0.001288,0.000000,0.000000,AdaBoostClassifier(),189,float,"{'base_estimator': AdaBoostClassifier(), 'n_es...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,60
1,15.199114,0.054027,2.347370,0.016289,AdaBoostClassifier(),116,auto,"{'base_estimator': AdaBoostClassifier(), 'n_es...",0.781891,0.806603,0.782368,0.775883,0.802661,0.789881,0.012322,30
2,0.004379,0.000160,0.000000,0.000000,AdaBoostClassifier(),30,float,"{'base_estimator': AdaBoostClassifier(), 'n_es...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,38
3,0.004572,0.000034,0.000000,0.000000,GradientBoostingClassifier(),84,float,{'base_estimator': GradientBoostingClassifier(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39
4,55.171960,0.114343,0.782275,0.018310,GradientBoostingClassifier(),126,auto,{'base_estimator': GradientBoostingClassifier(...,0.793200,0.815365,0.784953,0.787191,0.809546,0.798051,0.012206,7


In [38]:
results.sort_values(by='mean_test_score', ascending=False, inplace=True)
results.reset_index(drop=True, inplace=True)
results[[
    'param_base_estimator', 'param_n_estimators', 'param_sampling_strategy',
    'mean_test_score', 'std_test_score', 'rank_test_score'
]].head()

,param_base_estimator,param_n_estimators,param_sampling_strategy,mean_test_score,std_test_score,rank_test_score
0,GradientBoostingClassifier(),98,auto,0.798117,0.012181,1
1,GradientBoostingClassifier(),113,auto,0.798109,0.012196,2
2,GradientBoostingClassifier(),113,auto,0.798109,0.012196,2
3,GradientBoostingClassifier(),120,auto,0.798081,0.012211,4
4,GradientBoostingClassifier(),90,auto,0.798061,0.012227,5


In [43]:
X_train_preds = search.predict_proba(X_train)[:,1]
X_test_preds = search.predict_proba(X_test)[:,1]

print('Train roc_auc: ', roc_auc_score(y_train, X_train_preds))
print('Test roc_auc: ', roc_auc_score(y_test, X_test_preds))

Train roc_auc:  0.8198223065247972
Test roc_auc:  0.8080442775198536


By tuning hyperparameters of EasyEnsembleClassifier, we got slightly improved performance of the model. 